In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
sample = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
sample.head()

In [ ]:
with open("/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt","r") as file:
    content = file.read()
print(content)

In [ ]:
train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
train.head()

In [ ]:
test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")
test.head()

In [ ]:
train.info()

In [ ]:
train['Alley'].fillna("No alley", inplace=True)
train['MasVnrType'].fillna("No msr done", inplace=True)
train = train.drop('Id', axis=1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
print(train['SalePrice'].describe())
plt.figure(figsize=(9,8))
sns.distplot(train['SalePrice'], color='b', bins=100, hist_kws={'alpha': 0.4})

In [ ]:
list(set(train.dtypes.tolist()))

In [ ]:
numerical_representation = train.select_dtypes(include=['int64','float64'])
numerical_representation.head()
categorical_representation = train.select_dtypes(include=['O'])
train.head()

In [ ]:
categorical_columns = categorical_representation.columns.tolist()
train = pd.get_dummies(train, columns=categorical_columns, drop_first=True)
train.head()

In [ ]:
train.hist(figsize=(16,20), bins=40, xlabelsize=8, ylabelsize=8);

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state=42)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_tensor = torch.from_numpy(X_train_scaled).float()
y_train_tensor = torch.from_numpy(y_train.values).float()
X_test_tensor = torch.from_numpy(X_test_scaled).float()
y_test_tensor = torch.from_numpy(y_test.values).float()
y_train_tensor = y_train_tensor.view(-1,1)
y_test_tensor = y_test_tensor.view(-1, 1)

In [ ]:
print(X.shape)
print(y.shape)
print(X_train_scaled.shape)
print(X_test_scaled.shape)
print(y_train.shape)
print(y_test.shape)
print(y_train_tensor.shape)
print(y_test_tensor.shape)

In [ ]:
%debug
class LinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_dimension,1)
        
    def forward(self,x):
        return self.linear(x)
    
input_dimension = X_train_tensor.shape[1]
model = LinearRegression()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
epochs = 50
for epoch in range(epochs):
    y_pred = model(X_train_tensor)
    
    loss = criterion(y_pred, y_train_tensor)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 100 == 0:
        print(f"Epoch[{epoch+1}/{epochs}], Loss: {loss.item():.4f}")
model.eval()
with torch.no_grad():
    y_pred_test = model(X_test_tensor)
    test_loss = criterion(y_pred_test, y_test_tensor)
    print(f"Test loss: {test_loss.item():.4f}")


In [ ]:
test.info()